<a href="https://colab.research.google.com/github/Mahikarimib/Image-Classification/blob/main/data_utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import os
import glob
import shutil
import random
from tqdm import tqdm


def create_validation_data(trn_dir, val_dir, split=0.1):
   if not os.path.exists(val_dir):
      os.mkdir(val_dir)

   train_ds = glob.glob(trn_dir + '/*/*.png')    
   print(len(train_ds))   
    
   valid_sz = int(split * len(train_ds)) if split < 1.0 else split        
   valid_ds = random.sample(train_ds, valid_sz)  
   print(len(valid_ds))   

     
   for fname in tqdm(valid_ds):      
       basename = os.path.basename(fname)       
       label = fname.split('\\')[-2]       
       src_folder = os.path.join(trn_dir, label)         
       tgt_folder = os.path.join(val_dir, label)       
       if not os.path.exists(tgt_folder):           
          os.mkdir(tgt_folder)        
       shutil.move(os.path.join(src_folder, basename), os.path.join(tgt_folder, basename))


def pseudo_label(probs, tst_dir, test_dl, class_names, threshold=0.99999):    
    num_data = len(test_dl.dataset)
    preds = np.argmax(probs, axis=1)    
    candidate_idxs = np.arange(num_data)[probs.max(axis=1) >= threshold]        
    fnames = [f[0].split('\\')[-1] for f in test_dl.dataset.imgs]   
    imgs = [fnames[i] for i in candidate_idxs]  
    labels = [class_names[preds[i]] for i in candidate_idxs]     
    dest_folder = os.path.join(DATA_DIR, 'pseudo', 'train')
#     for name in class_names:
#         folder = os.path.join(dest_folder, name)
#         if not os.path.exists(folder):
#             os.mkdir(folder)            
    for _, (img, label) in tqdm(enumerate(zip(imgs, labels))):
        src = os.path.join(tst_dir, 'unk', img)
        dst = os.path.join(dest_folder, label, img)
        if not os.path.exists(dst):
            shutil.copyfile(src, dst)